# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [1]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [2]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "proceeding": {
        "file" : "../publications.bib",
        "key" : "inproceedings",
        "venuekey": "booktitle",
        "venue-pretext": "",
        "collection" : {
            "name":"publications",
            "permalink":"/publication/"
        }
    },
    "journal": {
        "file" : "../publications.bib",
        "key" : "article",
        "venuekey": "journal",
        "venue-pretext": "",
        "collection" : {
            "name":"publications",
            "permalink":"/publication/"
        }
    },
}

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [18]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    key = publist[pubsource]["key"]

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields

        try:
            pub_type = bibdata.entries[bib_id].type
            if pub_type.lower() != key:
                continue

            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""

            #citation authors - todo - add highlighting for primary author?
            authors = []
            for author in bibdata.entries[bib_id].persons["author"]:
                authors.append(author.first_names[0]+" "+author.last_names[0])
            citation = citation + ", ".join(authors) + ". "

            #citation title
            citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""

            #add venue logic depending on citation type
            venue = publist[pubsource]["venue-pretext"]+b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")

            citation = citation + " " + html_escape(venue)
            citation = citation + ", " + pub_year + "."

            
            ## YAML variables
            md = "---\ntitle: \""   + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + '"\n'
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\npaperurl: '" + b["url"] + "'"
                    url = True

            md += "\ncitation: '" + html_escape(citation) + "'"
            md += "\nexcerpt: ''"
            md += "\n---"

            
            ## Markdown description for individual page
            if url:
                md += "\n[Paper](" + b["url"] + "){:target=\"_blank\"}\n" 
        
            md += "\n\nCite as: \n```bibtex\n" + bibdata.entries[bib_id].to_string("bibtex") + "```"


            md_filename = os.path.basename(md_filename)

            with open("../_publications/" + md_filename, 'w') as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue

SUCESSFULLY PARSED robinson2014sparse: " Sparse Local Binary Pattern Histograms for Face Recognition  ... "
SUCESSFULLY PARSED jain2016: " An Approach to Integrate Inter-dependent Simulations using H ... "
SUCESSFULLY PARSED robinson2016: " Network Optimization of Food Flows in the U.S.  "
SUCESSFULLY PARSED robinson2017deep: " A Deep Learning Approach for Population Estimation from Sate ... "
SUCESSFULLY PARSED gupta2018infrastructure: " Infrastructure Resilience for Climate Adaptation  "
SUCESSFULLY PARSED robinson2018machine: " A Machine Learning Approach to Modeling Human Migration  "
SUCESSFULLY PARSED robinson2018toward: " Toward a Common Object Model for Integrated Transportation a ... "
SUCESSFULLY PARSED malkin2019label: " Label Super-Resolution Networks  "
SUCESSFULLY PARSED robinson2019large: " Large Scale High-Resolution Land Cover Mapping with Multi-Re ... "
SUCESSFULLY PARSED robinson2020human: " Human-Machine Collaboration for Fast Land Cover Mapping  "
SUCESSFULLY PARSE

In [13]:
print(bibdata.entries[bib_id].to_string("bibtex"))

@article{robinson2024seeing,
    author = "Robinson, Caleb and Corley, Isaac and Ortiz, Anthony and Dodhia, Rahul and Ferres, Juan M Lavista and Najafirad, Peyman",
    title = "Seeing the Roads Through the Trees: A Benchmark for Modeling Spatial Dependencies with Aerial Imagery",
    journal = "arXiv preprint arXiv:2401.06762",
    year = "2024",
    url = "https://arxiv.org/abs/2401.06762"
}

